In [1]:
import tensorflow as tf
import numpy as np

In [2]:
train_x = np.loadtxt('/Users/babburi/datasets/HAR/train/X_train.txt')
train_y = np.loadtxt('/Users/babburi/datasets/HAR/train/y_train.txt')
train_y = np.eye(6)[train_y.astype('int')-1] # one-hot encoding
test_x = np.loadtxt('/Users/babburi/datasets/HAR/test/X_test.txt')
test_y = np.loadtxt('/Users/babburi/datasets/HAR/test/y_test.txt')
test_y = np.eye(6)[test_y.astype('int')-1]

seed = 456
np.random.seed(seed)
np.random.shuffle(train_x)
np.random.seed(seed)
np.random.shuffle(train_y)
np.random.seed(seed)
np.random.shuffle(test_x)
np.random.seed(seed)
np.random.shuffle(test_y)

In [3]:
n_nodes_input = 561
n_nodes_hl = 30
n_classes = 6
x = tf.placeholder('float', [None, 561])
y = tf.placeholder('float')
#global_step = tf.Variable(0, name='global_step', trainable=False)
initial_lr = 0.1

In [11]:
def neural_network_model(data):
    # define weights and biases for all each layer
    hidden_layer = {'weights':tf.Variable(tf.truncated_normal([n_nodes_input, n_nodes_hl], stddev=0.3)),
                      'biases':tf.Variable(tf.constant(0.1, shape=[n_nodes_hl]))}
    output_layer = {'weights':tf.Variable(tf.truncated_normal([n_nodes_hl, n_classes], stddev=0.3)),
                    'biases':tf.Variable(tf.constant(0.1, shape=[n_classes]))}
    # feed forward and activations
    l1 = tf.add(tf.matmul(data, hidden_layer['weights']), hidden_layer['biases'])
    l1 = tf.nn.sigmoid(l1)
    output = tf.matmul(l1, output_layer['weights']) + output_layer['biases']
    
    return output

In [16]:
def train_neural_network(x):
    #lr = tf.train.exponential_decay(initial_lr, global_step, 1000, 0.96, staircase=True)
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=prediction))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    for epoch in range(1000):
        loss = 0
        _, c = sess.run([optimizer, cost], feed_dict = {x: train_x, y: train_y})
        loss += c
        if (epoch % 100 == 0 and epoch != 0):
            print('Epoch', epoch, 'completed out of', 1000, 'Training loss:', loss)
    correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='op_accuracy')
    
    print('Test set Accuracy:', sess.run(accuracy, feed_dict = {x: test_x, y: test_y}))

In [17]:
train_neural_network(x)

Epoch 100 completed out of 1000 Training loss: 0.38831833005
Epoch 200 completed out of 1000 Training loss: 0.311929166317
Epoch 300 completed out of 1000 Training loss: 0.261933743954
Epoch 400 completed out of 1000 Training loss: 0.217232555151
Epoch 500 completed out of 1000 Training loss: 0.180498018861
Epoch 600 completed out of 1000 Training loss: 0.151680067182
Epoch 700 completed out of 1000 Training loss: 0.123882032931
Epoch 800 completed out of 1000 Training loss: 0.101448066533
Epoch 900 completed out of 1000 Training loss: 0.0848900973797
Test set Accuracy: 0.951815
